In [1]:
# load our graph data from drive, if you run it locally, ignore this
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install networkx

In [62]:
import pickle
import numpy as np
import scipy as sp
import networkx
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch import nn

In [3]:
file = open('/content/drive/MyDrive/df_agg (1).pkl','rb')
df_agg = pickle.load(file)
df_agg.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 17544 entries, 2012-01-01 00:00:00 to 2013-12-31 23:00:00
Columns: 5558 entries, 2 to 5564
dtypes: float64(5558)
memory usage: 744.1 MB


In [4]:
file = open('/content/drive/MyDrive/windows_agg_ids (2).pkl','rb')
windows = pickle.load(file)
windows.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5558 entries, 0 to 5557
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype          
---  ------       --------------  -----          
 0   LCLid        5558 non-null   int64          
 1   Enabled At   5558 non-null   datetime64[ns] 
 2   Disabled At  5558 non-null   datetime64[ns] 
 3   Duration     5558 non-null   timedelta64[ns]
dtypes: datetime64[ns](2), int64(1), timedelta64[ns](1)
memory usage: 217.1 KB


# Filter nodes that are active from 2012-11-01(7320) to 2013-11-01(16079)

In [59]:
from tqdm import tqdm

In [60]:
timestampidx = 7320
active_nodes = df_agg.columns[df_agg.loc[df_agg.index[timestampidx], :].notna()].to_numpy()
for timestampidx in tqdm(range(7321, 16080)):
  act = df_agg.columns[df_agg.loc[df_agg.index[timestampidx], :].notna()].to_numpy()
  active_nodes = np.intersect1d(act, active_nodes)


100%|██████████| 8759/8759 [00:49<00:00, 177.68it/s]


In [61]:
len(active_nodes)

3708

Hence we only have 3708 nodes that are active from all timestamp in this period. If we would like more nodes, we could make interpolations.

# Data preparation

# MGTNN

paper:https://arxiv.org/abs/2005.11650

code: https://github.com/nnzhan/MTGNN/tree/master

A interesting framework which uses time-series data to make predictions.
The most impotant part is that this framework learn both the graph structure and GNN. That is, in their data they don't have a predefined graph structure at  first (like us), so they have a graph learning layer which learns the adjacency matrix from the time-series data. The learned adjacency matrix then is input to the graph convolution layer and temporal layer.

# Graph learning layer

In their work they choose to learn directed graph.
Code below shows the example an un-directed graph learning layer.

In [ ]:
class graph_undirected(nn.Module):
    def __init__(self, nnodes, k, dim, device, alpha=3, static_feat=None):
        super(graph_undirected, self).__init__()
        self.nnodes = nnodes
        if static_feat is not None:
            xd = static_feat.shape[1]
            self.lin1 = nn.Linear(xd, dim)
        else:
            self.emb1 = nn.Embedding(nnodes, dim)
            self.lin1 = nn.Linear(dim,dim)

        self.device = device
        self.k = k
        self.dim = dim
        self.alpha = alpha
        self.static_feat = static_feat

    def forward(self, idx):
        if self.static_feat is None:
            nodevec1 = self.emb1(idx)
            nodevec2 = self.emb1(idx)
        else:
            nodevec1 = self.static_feat[idx,:]
            nodevec2 = nodevec1

        nodevec1 = torch.tanh(self.alpha*self.lin1(nodevec1))
        nodevec2 = torch.tanh(self.alpha*self.lin1(nodevec2))

        a = torch.mm(nodevec1, nodevec2.transpose(1,0))
        adj = F.relu(torch.tanh(self.alpha*a))
        mask = torch.zeros(idx.size(0), idx.size(0)).to(self.device)
        mask.fill_(float('0'))
        s1,t1 = adj.topk(self.k,1)
        mask.scatter_(1,t1,s1.fill_(1))
        adj = adj*mask
        return adj


**Thought**: if we do not learn adj matrix in the framework, is it ok if we build the adj matrix from the similarity between time series data (maybe computation expensive)

# GCN and TCN
For GCN layer they use mix hop propogation.
For TCN layer they use dilated inception layer.
Those two are fancy but may not easy to use. We could choose simplier one.